In [1]:
import pprint
from scapy.all import *
from scapy.config import Conf
from scapy.layers.inet import IP, TCP, UDP
from scapy.layers.http import HTTP
from scapy.utils import hexdump
# Conf.use_bpf = False

In [2]:
def get_net_ifaces():
    # interfaces = [_ for _ in ifaces] # 兼容性有问题，Windows 不可用
    interfaces = [_ for _ in get_working_ifaces()]
    return interfaces

In [3]:
def analyze_packet(packet):
    if packet.haslayer(IP):
        ip = packet[IP]
        print("[*] Protocol: IP")
        print("\t Src IP:", ip.src, ' Dst IP:', ip.dst)
    
    if packet.haslayer(TCP):
        tcp = packet[TCP]
        print("[*] Protocol: TCP")
        print("\t Src Port:", tcp.sport, ' Dst Port:', tcp.dport)
        
    if packet.haslayer(UDP):
        udp = packet[UDP]
        print("[*] Protocol: UDP")
        print("\t Src Port:", udp.sport, ' Dst Port:', udp.dport)
    
    if packet.haslayer(HTTP):
        http = packet[HTTP]
        print("[*] Protocol: HTTP")
        print("\t Host:", http.host, ' Path:', http.path)
        
    rawhex = hexdump(packet, dump=True)
    print("[*] Raw:")
    print(rawhex)
    

In [ ]:
class Sniffer():
    def __init__(self):
        self.available_ifaces = get_net_ifaces()
        self.iface = None
        self.stop = False
        self.cached_packets = []
        self.max_cache_packets_count = 1

    def get_interfaces(self):
        for interface in self.available_ifaces:
            print(interface.name)
        return self.available_ifaces

    def set_interface(self, interface_name):
        for interface in self.available_ifaces:
            if interface_name == interface.name:
                self.iface = interface
                return True
        return False
        
        # if interface in self.available_ifaces:
        #     self.iface = interface
        #     return True
        # return False

    def start_sniffing(self):
        if self.iface == None:
            return False
        cached_packets = sniff(iface=self.iface, count=self.max_cache_packets_count, prn=self.print_callback) # stop_filter=self.stop_filter_callback
        self.cached_packets.extend(cached_packets)
        return True

    def print_callback(self, packet):
        print(packet.summary())

    def stop_filter_callback(self, packet):
        if self.stop:
            return True
        return False

    def pause(self):
        self.stop = True
        
    def reset(self):
        self.stop = False
        self.iface = None
        self.cached_packets = []
        
    def analyze_packets(self):
        for packet in self.cached_packets:
            print("[*] analysis:")
            print(packet)
            analyze_packet(packet)

In [ ]:
sniffer = Sniffer()

In [ ]:
sniffer.get_interfaces()

本地连接* 10
本地连接* 9
本地连接* 8
蓝牙网络连接
WLAN
VMware Network Adapter VMnet8
VMware Network Adapter VMnet1
vEthernet (Default Switch)
Clash
\Device\NPF_Loopback
以太网


[<NetworkInterface_Win WAN Miniport (Network Monitor) [UP+RUNNING+OK]>,
 <NetworkInterface_Win WAN Miniport (IPv6) [UP+RUNNING+OK]>,
 <NetworkInterface_Win WAN Miniport (IP) [UP+RUNNING+OK]>,
 <NetworkInterface_Win Bluetooth Device (Personal Area Network) [UP+RUNNING+WIRELESS+DISCONNECTED]>,
 <NetworkInterface_Win Intel(R) Wi-Fi 6 AX200 160MHz [UP+RUNNING+WIRELESS+OK]>,
 <NetworkInterface_Win VMware Virtual Ethernet Adapter for VMnet8 [UP+RUNNING+OK]>,
 <NetworkInterface_Win VMware Virtual Ethernet Adapter for VMnet1 [UP+RUNNING+OK]>,
 <NetworkInterface_Win Hyper-V Virtual Ethernet Adapter [UP+RUNNING+OK]>,
 <NetworkInterface_Win Clash Tunnel [UP+RUNNING+OK]>,
 <NetworkInterface_Win Software Loopback Interface 1 [LOOPBACK+UP+RUNNING+OK+DISCONNECTED]>,
 <NetworkInterface_Win Realtek PCIe 2.5GbE Family Controller [UP+RUNNING+DISCONNECTED]>]

In [ ]:
sniffer.set_interface('Clash')

True

In [ ]:
sniffer.start_sniffing()